<a href="https://colab.research.google.com/github/hppranaav02/bad-poets-society/blob/main/bad_poets_society.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate poems though a creative statistical or stochaistic process

After going through a few styles of poems we have settled on sticking to poems from the Nobel Laureate William Butler Yeats for their simple rhyme scheme and broad topics about life, and sonnets from William shakespeare for complexity in wording, structure and topics like love and passion (He was probably the Elvis Presley of his time).

We came up with these choices partly because one of the group members studied Shakespeares' work [As you like it](https://www.shakespeare.org.uk/explore-shakespeare/shakespedia/shakespeares-plays/as-you-like-it/) in high school (_Side Note: It was fun but excrutiatingly intense_). During our conversations, he came up with the absurd idea of bringing Shakespeare to modern times and making him learn modern english. All to see what kind of literature he would produce with the present language.

We want our system to generate a shakespearean style verse with a modern twist to it. We are still not sure how we will achieve this but we will take a simple approach first (maybe explore RiTa a bit) to get a feel.  

Some of the poems which we were motivated by are as follows

W. B. Yeats

* [Sailing to Byzantium](https://colab.research.google.com/drive/1eanny4nzn0HxlPce8Y1mpJ9__HsHdjRx#scrollTo=vvU3S2VwNfB7&line=7&uniqifier=1)

* [The Stolen Child](https://www.poemhunter.com/poem/the-stolen-child/)

* [A Prayer for my daughter](https://www.poemhunter.com/poem/a-prayer-for-my-daughter/)

W. Shakespeare

* [Sonnet 130 - "My mistress' eye are nothing like the sun"](https://www.poemhunter.com/poem/sonnet-130-my-mistress-eyes-are-nothing-like-the/)

* [Sonnet 18 - "Shall I compare thee to a summers day"](https://www.poemhunter.com/poem/sonnet-18-shall-i-compare-thee-to-a-summer-s-day/)

* [Sonnet 104 - To me, Fair friend, You can never be old](https://www.poemhunter.com/poem/sonnet-104-to-me-fair-friend-you-never-can-be-old/)


 Right now we are evaluating how generator systems work. Markov Chains, we thought would be the right first step to try and generate poems given its simplicity.

  * Markov Chains: Using RiTa with Java, we were able to get an idea of how markov chains produce output based on the inputs given to it. In our example we used poems from W.B. Yeats and sonnets from William Shakespeare. Our hopes were that we would be able to generate poems which  blend elizabethian and modern english to create an illusion of what shakespeare would write if he time travelled to the modern world and learnt modern english.

  * We varied the temprature settings to pick up infrequent words with higher probability to see the results but that resulted in unusual outputs with no coherence.

  * Based on our results, the output did not generate the text exactly what we were looking for. But the form of speech was good without structure. An example of the text generated by our Markov chains are:


    (2-gram)
    Mere dreams - and my verse alone did decree That then,
    While up, bodies broken rings Upon the light, for thy dear perpetual dulness.
    And what good; Wherefore I dropped the moon may deserve to the heights of my soul
    Does Minnaloushe runs to work of his beams assemble?
    The first of thralled discontent, I, The pacing to that men as sweet-season'd showers
    There lives th 'eyes, and poets can understand.

    (3-gram)
    The mortal moon hath her wish, thou mine, mine eyes best see, Despite of wrinkles, this thy golden time.
    What merit do I not spend Revenge upon myself, but with divining eyes, seeing this, let him bring forth Eternal numbers to outlive long date.
    Then can I grieve at grievances foregone, And stars began to peep.
    This other man I had dreamed A drunken, vainglorious lout.
    Excuse not silence so, To swear against the cold, Bare ruined choirs, where thou art?

* We have decided to use a neural network, specifically a Recurrent Neural Network (RNN). A recurrent neural network fits well for text generation as its structure defines it to uses information persistence to aloow information to be passed from one step to another. Normal neural networks have this exact shortcoming.

  * In RNNs, a special neural network which uses long short term memory, called LSTM, which we have decided to explore. As a RNN, LSTM have the same structure of self looping nodes with an addition of a cell state which helps pass information through its iterations.

  * Tensorflow - This is required to import the LSTM neural network model and other tools to help format data
  * Pandas - Used to pre-process data which is then supplied to the neural network

  * For our data, it was initially just a set of poems from W.B. Yeats totalling 3000 words. This caused some nice outputs but the model broke down at about 30 words after which words would repeat often

  * Update: By increasing the epochs from 50 to 100 resulted in better text generation. In terms of creativity

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

data = open('sample_data/poems.txt',encoding="utf-8").read();

# Generating the corpus by
# splitting the text into lines
corpus = data.lower().split("\n")

# get tokens out from lines
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

# Vocabulary count of the corpus
total_words = len(tokenizer.word_index)

# Converting the text into embeddings
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]

    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,
                                         maxlen=max_sequence_len,
                                         padding='pre'))
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = ku.to_categorical(label, num_classes=total_words+1)


# Building a Bi-Directional LSTM Model
model = Sequential()
model.add(Embedding(total_words+1, 100,
                    input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words+1/2, activation='relu',
                kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words+1, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
print(model.summary())


#train model
history = model.fit(predictors, label, epochs=100, verbose=1)


Total Words: 2621
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 12, 100)           262200    
                                                                 
 bidirectional (Bidirection  (None, 12, 300)           301200    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 12, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 2621)              264721    
                                                                 
 dense_1 (Dense)             (None, 2622)              6874884   
                                      

In [ ]:
seed_text = "This is a flow, i like to grow"
next_words = 50 - len(seed_length)
ouptut_text = ""

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1,
      padding='pre')
    predicted = np.argmax(model.predict(token_list,
                                        verbose=0), axis=-1)
    output_word = ""

    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

    seed_text += " " + output_word

print(seed_text)

This is a flow, i like to grow enamelling drowned her mirth right and heaven can not the summer innocence of years and share enamelling loss wed enamelling plain you sleep and a fairy queen that rose with one or stricken deaf and dumb and blind at grief flat and wattles do enamelling enamelling loss loss were the
